# Dependencies

For ZnTrack there are two different ways to set up dependencies:

1. Node/Stage based dependencies
2. File based dependencies

## Node dependencies
We will first look at Node based dependencies starting from a RandomNumber `Hello World` example.
In our first stage we create a random number and then we add another Node that depends on this one.
We can do this very easily by setting the `dvc.deps` in the following way
```py
dependency: Stage = dvc.deps(Stage.load()
```

This allows us to access all properties of the `dependency` attribute.

In [1]:
from zntrack import Node, dvc, zn, config
from random import randrange
from pathlib import Path

In [2]:
config.nb_name = "03_dependencies.ipynb"

In [3]:
from zntrack.utils import cwd_temp_dir

temp_dir = cwd_temp_dir()

In [4]:
!git init
!dvc init

Initialized empty Git repository in C:/Users/fabia/AppData/Local/Temp/tmpy4v49p74/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [5]:
class RandomNumber(Node):
    maximum = zn.params()
    number = zn.outs()

    def __init__(self, maximum=None, **kwargs):
        super().__init__(**kwargs)
        self.maximum = maximum

    def run(self):
        self.number = float(randrange(self.maximum))


class ComputePower(Node):
    random_number: RandomNumber = dvc.deps(RandomNumber.load())
    number = zn.outs()
    power = zn.params()

    def __init__(self, power=2, **kwargs):
        super().__init__(**kwargs)
        self.power = power

    def run(self):
        self.number = self.random_number.number**self.power

We can now create the stages the usual way and look at the outcomes

In [6]:
RandomNumber(maximum=16).write_graph()
ComputePower(power=2.0).write_graph()

2022-02-18 09:54:46,088 dvcgraph (WARNING): --- Writing new DVC file! ---
2022-02-18 09:54:46,089 jupyter (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-02-18 09:54:46,089 jupyter (WARNING): Converting 03_dependencies.ipynb to file RandomNumber.py
2022-02-18 09:54:50,092 utils (INFO): Creating 'dvc.yaml'
Adding stage 'RandomNumber' in 'dvc.yaml'

To track the changes with git, run:

    git add dvc.yaml 'nodes\RandomNumber\.gitignore'

To enable auto staging, run:

	dvc config core.autostage true

2022-02-18 09:54:50,097 dvcgraph (WARNING): --- Writing new DVC file! ---
2022-02-18 09:54:50,098 jupyter (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-02-18 09:54:50,098 jupyter (WARNING): Converting 03_dependencies.ipynb to file ComputePower.py
2022-

In [7]:
!dvc repro

Running stage 'RandomNumber':
> python -c "from src.RandomNumber import RandomNumber; RandomNumber.load(name='RandomNumber').run_and_save()" 
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

Running stage 'ComputePower':
> python -c "from src.ComputePower import ComputePower; ComputePower.load(name='ComputePower').run_and_save()" 
Updating lock file 'dvc.lock'

To track the changes with git, run:

    git add dvc.lock

To enable auto staging, run:

	dvc config core.autostage true
Use `dvc push` to send your updates to remote storage.


In [8]:
print(
    f"{RandomNumber.load().number} ^ {ComputePower.load().power} ="
    f" {ComputePower.load().number}"
)

9.0 ^ 2.0 = 81.0


## File dependencies
The second approach is depending on files.
We do this by creating a file from our random number first.
We then use the path to that file as our dependency.
In the end we compare both methods and also show the graph that was built for us by DVC.

We can do set the file dependency very easily in the same way by passing `pathlib.Path` or `str` to the `dvc.deps`.
As every  `dvc.<...>` it also supports lists.
```py
dependency: Path = dvc.deps([Path('some_file.txt'), 'some_other_file.txt'])
```

If there are now arguments to `load` we can also use `dvc.deps(RandomNumber)` instead of `dvc.deps(RandomNumber.load())`. This should not be used with attributes such as  `dvc.deps(RandomNumber.file)`, because the file name could change and this would not be recognized. In general, for Node dependecies the full Node should be passed and not some Node attributes to avoid potentially unwanted behaviour.

In [9]:
class WriteToFile(Node):
    random_number: RandomNumber = dvc.deps(RandomNumber)
    file: Path = dvc.outs(Path("random_number.txt"))

    def run(self):
        self.file.write_text(str(self.random_number.number))


class PowerFromFile(Node):
    file: Path = dvc.deps(WriteToFile.load().file) # try to avoid this
    number = zn.outs()
    power = zn.params(2)

    def run(self):
        number = float(self.file.read_text())
        self.number = number**self.power


class ComparePowers(Node):
    # both are possible, ComputePower or ComputePower.load()
    power_deps = dvc.deps([PowerFromFile.load(), ComputePower])

    def run(self):
        assert self.power_deps[0].number == self.power_deps[1].number

In [10]:
WriteToFile().write_graph()
PowerFromFile().write_graph()
ComparePowers().write_graph()

2022-02-18 09:54:58,136 dvcgraph (WARNING): --- Writing new DVC file! ---
2022-02-18 09:54:58,137 jupyter (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-02-18 09:54:58,137 jupyter (WARNING): Converting 03_dependencies.ipynb to file WriteToFile.py
2022-02-18 09:55:02,285 utils (INFO): Adding stage 'WriteToFile' in 'dvc.yaml'

To track the changes with git, run:

    git add .gitignore dvc.yaml

To enable auto staging, run:

	dvc config core.autostage true

2022-02-18 09:55:02,286 dvcgraph (WARNING): --- Writing new DVC file! ---
2022-02-18 09:55:02,287 jupyter (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-02-18 09:55:02,287 jupyter (WARNING): Converting 03_dependencies.ipynb to file PowerFromFile.py
2022-02-18 09:55:06,256 utils (INFO): Adding st

In [11]:
!dvc dag

                +--------------+                  
                | RandomNumber |                  
                +--------------+                  
                **             ***                
             ***                  ***             
           **                        **           
 +-------------+                       **         
 | WriteToFile |                        *         
 +-------------+                        *         
        *                               *         
        *                               *         
        *                               *         
+---------------+               +--------------+  
| PowerFromFile |               | ComputePower |  
+---------------+               +--------------+  
                **             ***                
                  ***        **                   
                     **    **                     
                +---------------+                 
                | ComparePowers

In [12]:
!dvc repro

Stage 'RandomNumber' didn't change, skipping
Stage 'ComputePower' didn't change, skipping
Running stage 'WriteToFile':
> python -c "from src.WriteToFile import WriteToFile; WriteToFile.load(name='WriteToFile').run_and_save()" 
Updating lock file 'dvc.lock'

Running stage 'PowerFromFile':
> python -c "from src.PowerFromFile import PowerFromFile; PowerFromFile.load(name='PowerFromFile').run_and_save()" 
Updating lock file 'dvc.lock'

Running stage 'ComparePowers':
> python -c "from src.ComparePowers import ComparePowers; ComparePowers.load(name='ComparePowers').run_and_save()" 
Updating lock file 'dvc.lock'

To track the changes with git, run:

    git add dvc.lock

To enable auto staging, run:

	dvc config core.autostage true
Use `dvc push` to send your updates to remote storage.


In [13]:
# to verify we can also run the method manually
ComparePowers.load().run()

If we now look at our `dvc.yaml` we can see that for our Node dependencies we rely on the `outs/<node_name>.json` while for the file dependency it is directly connect to the passed file.

In [14]:
from IPython.display import Pretty, display

display(Pretty("dvc.yaml"))

stages:
  RandomNumber:
    cmd: "python -c \"from src.RandomNumber import RandomNumber; RandomNumber.load(name='RandomNumber').run_and_save()\"\
      \ "
    deps:
    - src/RandomNumber.py
    params:
    - RandomNumber
    outs:
    - nodes/RandomNumber/outs.json
  ComputePower:
    cmd: "python -c \"from src.ComputePower import ComputePower; ComputePower.load(name='ComputePower').run_and_save()\"\
      \ "
    deps:
    - nodes/RandomNumber/outs.json
    - src/ComputePower.py
    params:
    - ComputePower
    outs:
    - nodes/ComputePower/outs.json
  WriteToFile:
    cmd: "python -c \"from src.WriteToFile import WriteToFile; WriteToFile.load(name='WriteToFile').run_and_save()\"\
      \ "
    deps:
    - nodes/RandomNumber/outs.json
    - src/WriteToFile.py
    outs:
    - random_number.txt
  PowerFromFile:
    cmd: "python -c \"from src.PowerFromFile import PowerFromFile; PowerFromFile.load(name='PowerFromFile').run_and_save()\"\
      \ "
    deps:
    - random_number.txt
   

In [ ]:
temp_dir.cleanup()